In [3]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.facecolor'] = "white"

# Reservoir
Dynamics of reservoir are described by (continuous)
$$
\dot{\mathbf{r}}(t) = -\gamma \mathbf{r}(t) + \gamma \tanh(W_r \mathbf{r}(t) + W_{\text{in}}\mathbf{u}(t)).
$$
If we discretise time as $\{\tau_k\}_{k=1}^n$, with 
$\Delta \tau = \mathbf{r}(\tau_{k+1}) - \mathbf{r}(\tau_k)$, then we arrive at 
the discretised dynamics for the reservoir:
$$
\mathbf{r}(\tau_{n+1}) = \mathbf{r}(\tau_n) + \dot{\mathbf{r}}(\tau_n)\Delta \tau.
$$
Note that the size of this time step $\Delta \tau$ should be equal to the size of 
the time step of the dynamical system being forecasted. 

Dimensions:
- $N = 100$, $d = 3$
- reservoir vector $\mathbf{r}(t) \in \reals^N$
- input signal $\mathbf{u}(t) \in \reals^d$

which means that
- $W_r$ is $N \times N$ matrix, encoding connections between reservoir nodes in 
network
- $W_{\text{in}}$ is $N \times d$ matrix, encoding connections between input and
nodes within reservoir

and $\gamma \in \reals$ is a hyperparameter.

In [20]:
def next_res(r_prev: np.ndarray, u_prev: np.ndarray, hyperparams: dict, W_r: np.ndarray, W_in: np.ndarray, delta_t: np.double):
    """
    Computes the dynamics of the reservoir.

    Args:
        r_prev (np.ndarray): the previous reservoir state, of shape N x 1.
        u_prev (np.ndarray): the previous input signal, of shape d x 1.
        hyperparams (dict): dictionary of hyperparameters.
            Must have the key 'GAMMA'. A sensible range for GAMMA is 7 - 11. 
        W_r (np.ndarray): the adjacency matrix of the internal connection 
            network, of shape N x N. 
        W_in (np.ndarray): the adjacency matrix of the input signal to node 
            network, of shape N x d.  
        delta_t (np.double): the size of the time step. 
            Note that this should be the same as the time step used to integrate
            the dynamical system being forecasted (i.e. the input signal). 
    
    Returns:
        (np.ndarray): the next reservoir state, of shape N x 1. 
    """

    # parse hyperparameter
    GAMMA = hyperparams["GAMMA"]
    
    return r_prev + delta_t * ((-GAMMA) * r_prev) + GAMMA * np.tanh(np.dot(W_r, r_prev) + np.dot(W_in, u_prev))

## Internal Reservoir Construction
The hyperparameters are:
- $\sigma$, the probability that a reservoir node is connected to an input 
signal component
- $\rho_{\text{in}}$, the variance of the weight, if it exists, of the
connection between a reservoir node and an input signal component
- $k$, the in-degree of nodes in the internal connection network
- $\rho_{r}$, the spectral radius of the internal connection network

For $W_{\text{in}}$, think of it as an nd-array with shape $(N, d)$ (i.e. 
consider its adjacency matrix). 

1. For each cell in $W_{\text{in}}$, draw from Bernoulli($\sigma$). 1 represents
connected and 0 represents not connected. 
2. For each cell in $W_{\text{in}}$, if it is connected (i.e. value 1), then 
set it to a random value drawn from N(0, $\rho_{\text{in}}^2$). If the cell has 
value 0, then leave it as 0. 

For $W_r$, think of it as an nd-array with shape $(N, N)$ (i.e. consider its
adjacency matrix). The value in the $i$-th row and $j$-th column represents 
the directed, weighted edge going from node $j$ to node $i$. In other words, 
the number of nonzero entries in row $i$ is the in-degree of node $i$. 

1. For the $i$-th row ($i = 1,\dots,N$), choose $0 < k \leq N$ integers 
without replacement, denote using $n_1,\dots,n_k$. 
2. Set the entries in the $i$-th row and $n$-th column ($n = n_1,\dots,n_k$) to 
be some weight drawn from Normal(0,1). All other entries in the $i$-th row are 
set to 0. 
3. Scale $W_r$ so that its spectral radius is equal to $\rho_r$. The spectral 
radius of a matrix is defined as its largest absolute eigenvalue. 

In [6]:
def generate_W_in(hyperparams: dict, shape: tuple, seed: int) -> np.ndarray:
    """
    Given hyperparameters, generates the matrix W_in, representing connections 
    between the input signal and nodes within the reservoir. 

    Args:
        hyperparams (dict): dictionary of hyperparameters.
            Must have the keys 'SIGMA' and 'RHO_IN'. 
        shape (tuple): tuple of two nonnegative integers, representing N x d.
        seed (int): seed for use when sampling using numpy.
            Consider keeping track of the seed used to ensure results are 
            reproducible. 

    Returns:
        (np.ndarray): matrix of shape N x d.
    """
    
    # parse hyperparameters
    SIGMA = hyperparams['SIGMA']
    RHO_IN = hyperparams['RHO_IN']

    # initialise output matrix of given shape
    W_in = np.ndarray(shape)

    # set random state using given seed
    state = np.random.RandomState(seed)

    # fill in connections in W_in 
    for i in range(W_in.shape[0]):
        for j in range (W_in.shape[1]):
            # whether or not connection exists is Bernoulli(SIGMA)
            W_in[i][j] = (state.uniform() < SIGMA) * 1
            # if connection exists, its weight is Normal(0, RHO_IN ** 2)
            if W_in[i][j] == 1:
                W_in[i][j] = (state.normal(0, RHO_IN ** 2))

    return W_in


def generate_W_r(hyperparams: dict, shape: tuple, seed: int):
    """
    Given hyperparameters, generates the matrix W_r, representing connections
    between reservoir nodes in the network. 
    
    Args:
        hyperparams (dict): dictionary of hyperparameters. 
            Must have the keys 'K' and 'RHO_R'.
        shape (tuple): tuple of two nonnegative integers, representing N x N.
        seed (int): seed for use when sampling using numpy.
            Consider keeping track of the seed used to ensure results are 
            reproducible. 

    Returns:
        (np.ndarray): matrix of shape N x N.
    """

    # parse hyperparameters
    K = hyperparams["K"]
    RHO_R = hyperparams["RHO_R"]

    # initialise output matrix of given shape
    W_r = np.ndarray(shape)

    # set random state using given seed
    state = np.random.RandomState(seed)

    # fill in connections in W_r
    for i in range(W_r.shape[0]):
        # choose k nodes without replacement
        choices = state.choice(W_r.shape[0], size=K, replace=False)
        
        for j in range(W_r.shape[1]):
            if j in choices:
                W_r[i][j] = state.normal(0, 1)
            else:
                W_r[i][j] = 0

    # spectral radius of W_r
    spectral_radius = max(abs(np.linalg.eigvals(W_r))) 

    # rescale W_r so that its spectral radius is equal to RHO_R
    W_r = (RHO_R / spectral_radius) * W_r

    return W_r

In [11]:
hyperparams = {
    'GAMMA': 9,
    'SIGMA': 0.5,
    'RHO_IN': 1,
    'K': 3,
    'RHO_R': 1
}

N = 100
d = 3

In [8]:
generate_W_in(hyperparams=hyperparams, shape=(N, d), seed=42)

array([[-1.11188012,  0.31890218,  0.27904129],
       [ 0.        ,  1.01051528,  0.        ],
       [ 0.        , -0.46341769, -0.46572975],
       [ 0.        ,  0.81644508, -1.523876  ],
       [-0.7033438 ,  0.        , -2.13962066],
       [ 0.        ,  0.        , -1.15099358],
       [ 0.37569802,  0.        ,  0.        ],
       [ 0.        ,  0.291034  , -0.63555974],
       [-0.5336488 ,  0.        , -0.00552786],
       [ 0.        ,  2.77831304,  1.19363972],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [-1.00908534, -1.58329421,  0.        ],
       [ 0.34361829, -1.76304016,  0.        ],
       [ 0.58212279,  0.        ,  0.        ],
       [ 0.88774846, -1.50815329,  1.09964698],
       [ 1.17971634,  0.        ,  0.        ],
       [-0.89820794,  0.97296353,  0.        ],
       [ 0.        ,  0.79559546,  0.        ],
       [ 0.08704707, -0.29900735,  0.        ],
       [ 0.        ,  1.47789404, -0.518

In [9]:
generate_W_r(hyperparams=hyperparams, shape=(N, N), seed=42)

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.16228114],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , -1.09112552,  0.        , ...,  0.        ,
         0.        ,  0.        ]])